# Chapter 10: Spatial data management - raster formats

*Rasterio* is a highly useful module for raster processing which you can use for reading and writing several different raster formats in Python.

## How to install *rasterio*?

We need to install the rasterio package.

### Anaconda - Platform independent

If you have Anaconda installed, open the *Anaconda Prompt* and type in:
```
conda install -c conda-forge rasterio
```

### Python Package Installer (pip) - Linux

If you have standalone Python3 and Jupyter Notebook install on Linux, open a command prompt / terminal and type in:
```
pip3 install rasterio
```

### Python Package Installer (pip) - Windows

The installation of *rasterio* is much more complicated with *pip* on Windows, because it depends on the *GDAL* library, for which the binaries must be installed separately or compiled from source.
An easier approach is to install these packages from [Python binary wheel files](https://www.lfd.uci.edu/~gohlke/pythonlibs/).

Due to its complexity these options are only recommended for advanced Python users and it is **strongly advised to use Anaconda on Windows**.

## How to use *rasterio*?

The rasterio package is also a module which you can simply import.
```python
import rasterio
```

## Opening a dataset

The `open()` function takes a path string or path-like object and returns an opened dataset object.
The path may point to a file of any supported raster format.

The `LC08_L1TP_188027_20200420_20200508_01_T1.tif` file is a Landsat 8 satellite image from Budapest and parts of Western-Hungary, acquired on 2020 April 20.  
Download: https://gis.inf.elte.hu/files/public/landsat-budapest-2020 (1.4 GB)

In [ ]:
import rasterio
budapest_2020 = rasterio.open('LC08_L1TP_188027_20200420_20200508_01_T1.tif')

Dataset objects have some attributes regarding the opened file:

In [ ]:
print(budapest_2020.name)
print(budapest_2020.mode) # by default the file is opened in read mode
print(budapest_2020.closed) # will be True after closed() called

Properties of the raster data stored in the example GeoTIFF can be accessed through attributes of the opened dataset object.

In [ ]:
print(budapest_2020.count) # band count
print(budapest_2020.width) # dimensions
print(budapest_2020.height)

## Dataset georeferencing

A GIS raster dataset is different from an ordinary image; its elements (or “pixels”) are mapped to regions on the earth’s surface. All pixels of a dataset is contained within a spatial bounding box.

In [ ]:
print(budapest_2020.bounds)

Our example covers the world from 210285 meters to 449415 meters left to right, and 5135985 meters to 5378115 meters bottom to top. Therefore it covers a region 239.13 kilometers wide by 242.13 kilometers high.

The value of `bounds` attribute is derived from a more fundamental attribute: the dataset’s geospatial transform.

In [ ]:
print(budapest_2020.transform)

A dataset’s `transform` is an affine transformation matrix that maps pixel locations in *(row, col)* coordinates to *(x, y)* spatial positions. The product of this matrix and `(0, 0)`, the row and column coordinates of the upper left corner of the dataset, is the spatial position of the upper left corner.

In [ ]:
print(budapest_2020.transform * (0, 0))

The position of the lower right corner is obtained similarly.

In [ ]:
print(budapest_2020.transform * (budapest_2020.width, budapest_2020.height))

But what do these numbers mean? 209985 meters from where? These coordinate values are relative to the origin of the dataset’s coordinate reference system (CRS).

In [ ]:
print(budapest_2020.crs)

All metadata for the whole raster dataset can be displayed easily if desired:

In [ ]:
print(budapest_2020.meta)

## Reading raster data

Data from a raster band can be accessed by the band’s index number. Following the [GDAL](https://gdal.org/) convention (on which library Rasterio depends on), bands are indexed from 1.

In [ ]:
print(budapest_2020.indexes)

Landsat 8 satellite images contain 11 bands, in the following order:

| Band Number | Description               | Wavelength        | Resolution |
|-------------|---------------------------|-------------------|------------|
| Band 1      | Coastal / Aerosol         | 0.433 to 0.453 µm | 30 meter   |
| Band 2      | Visible blue              | 0.450 to 0.515 µm | 30 meter   |
| Band 3      | Visible green             | 0.525 to 0.600 µm | 30 meter   |
| Band 4      | Visible red               | 0.630 to 0.680 µm | 30 meter   |
| Band 5      | Near-infrared             | 0.845 to 0.885 µm | 30 meter   |
| Band 6      | Short wavelength infrared | 1.56 to 1.66 µm   | 30 meter   |
| Band 7      | Short wavelength infrared | 2.10 to 2.30 µm   | 60 meter   |
| Band 8      | Panchromatic              | 0.50 to 0.68 µm   | 15 meter   |
| Band 9      | Cirrus                    | 1.36 to 1.39 µm   | 30 meter   |
| Band 10     | Long wavelength infrared  | 10.3 to 11.3 µm   | 100 meter  |
| Band 11     | Long wavelength infrared  | 11.5 to 12.5 µm   | 100 meter  |

We can read the bands of a dataset with the `read()` method:

In [ ]:
red = budapest_2020.read(4)
green = budapest_2020.read(3)
blue = budapest_2020.read(2)

Bands are simply 2D mathematical matrixes stored as multi-dimensional *NumPy* arrays. [*NumPy*](https://numpy.org/) is a first-rate library for numerical programming. It is widely used in academia, finance and also in the industry.

Not only *Rasterio*, but the already introduced *Pandas* library (see [Chapter 7](07_tabular_solutions.ipnyb)) is also built on top of *NumPy*, providing high-performance, easy-to-use data structures and data analysis tools, making data manipulation and visualization more convinient.

In [ ]:
print(type(red))
print(red)

For a *NumPy* array we can easily get the range and the mean of the values:

In [ ]:
print(red.min())
print(red.max())
print(red.mean())

Values from the array can be addressed by their row, column index.

In [ ]:
print(red[2000, 2000]) # random position

## Plotting

Since *Rasterio* reads raster data into mathematical matrixes (*numpy arrays*), plotting a single band as two dimensional data can be accomplished directly with *matplotlib*, as it also strongly depends on *NumPy*.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.imshow(red)
plt.colorbar()
plt.show()

In our case the data is not evenly distributed in the range $[0, 65535]$, most values are below 20000. The maximum and minimum value for visualization can be overriden with the `vmax` and `vmin` parameters.

In [ ]:
plt.imshow(red, vmax=20000)
plt.colorbar()
plt.show()

Instead of using a static value (20000), calculate the 99.99% percentile of each bands to remove only the few outliers (0.01%) from visualization.

In [ ]:
import numpy as np

red_max = np.percentile(red, 99.99)
blue_max = np.percentile(blue, 99.99)
green_max = np.percentile(green, 99.99)
print(red_max)
print(blue_max)
print(green_max)

*Remark:* here we use the numpy package directly to calculate the 99.99% percentile. NumPy is a module which can be imported as usual and is aliased with the `np` abbreviation in most cases.

The `vmax` parameter can be defined as a dynamic value for the now:

In [ ]:
plt.imshow(red, vmax=red_max)
plt.colorbar()
plt.show()

Color maps can also be used with the `cmap` parameter (*see [Chapter 9](09_spatial_vector_solutions.ipynb) for more details*).

In [ ]:
plt.imshow(red, vmax=red_max, cmap='Reds')
plt.colorbar()
plt.show()

## Histogram

Create a histogram of the visible bands of the Landsat satellite image.

First, create a pandas *DataFrame* from the 3 bands. The *DataFrame* shall contain 3 *Series*: one for each band. The *DataFrame* shall contain as many rows as many pixels are in the image.
To achive this we *flatten* the 2D matrixes into 1D vectors. (For *NumPy* both of them are arrays, regardless of their dimensions.)

In [ ]:
print("2D array:")
print(red)
print("Type: {0}, Size: {1}".format(type(red), red.size))
print()
print("1D array:")
red_vector = red.flatten()
print(red_vector)
print("Type: {0}, Size: {1}".format(type(red), red.size))

In [ ]:
import pandas as pd

budapest_df = pd.DataFrame({
    'red': red.flatten(),    
    'blue': blue.flatten(),    
    'green': green.flatten()
})
display(budapest_df)
display(budapest_df.iloc[20000000]) # random row

Now we can create the histograms for the *Series* (or for the *DataFrame* to display it on a single plot), as we have learned it in [Chapter 8](08_plotting_solutions.ipnyb).

In [ ]:
budapest_df['red'].plot(kind='hist', bins=50, color='red', title='Histogram of the visible red band')
plt.show()
budapest_df['blue'].plot(kind='hist', bins=50, color='blue', title='Histogram of the visible blue band')
plt.show()
budapest_df['green'].plot(kind='hist', bins=50, color='green', title='Histogram of the visible green band')
plt.show()

This visually verifies our previous conclusion that most values for the visible colour bands are below 20000.

Get the histogram of the "interesting" part:

In [ ]:
budapest_df['red'].plot(kind='hist', bins=100, xlim=(0, 20000), color='red', title='Histogram of the visible red band')
plt.show()
budapest_df['blue'].plot(kind='hist', bins=100, xlim=(0, 20000), color='blue', title='Histogram of the visible blue band')
plt.show()
budapest_df['green'].plot(kind='hist', bins=100, xlim=(0, 20000), color='green', title='Histogram of the visible green band')
plt.show()

### Multi-band plotting

Rasterio also provides `rasterio.plot.show()` to perform common tasks such as displaying multi-band images as RGB and labeling the axes with proper geo-referenced extents.

It can be used for a single band:

In [ ]:
from rasterio.plot import show

show(red, vmax=red_max)
plt.show()

For multiple bands to visualize in a true-color image, the values must be in the range of $[0, 255]$ or in the float range of $[0, 1]$.

In [ ]:
# astype('f4') is a numpy function to convert to float (4 byte)
redf = red.astype('f4') / red_max
bluef = blue.astype('f4') / blue_max
greenf = green.astype('f4') / green_max
rgb = [redf, greenf, bluef]

In [ ]:
show(rgb)
plt.show()

Increase the figure size:

In [ ]:
plt.figure(figsize=[10,10])
show(rgb)
plt.show()

---

## Example computation: NDVI

The [Normalized Difference Vegetation Index](https://en.wikipedia.org/wiki/Normalized_difference_vegetation_index) is a simple indicator that can be used to assess whether the target includes healthy vegetation. This calculation uses two bands of a multispectral image dataset, the Red and Near-Infrared (NIR) bands.

In [ ]:
nir = budapest_2020.read(5)
plt.imshow(nir, vmax=2**15)
plt.colorbar()
plt.show()

In [ ]:
nir_max = np.percentile(nir, 99.99)
print(nir_max)
nirf = nir.astype('f4') / nir_max

The value of *NVDI* can be calculated with a simple mathematical formula:

$$NDVI = \frac{NIR - Red}{NIR + Red}$$

With Rasterio we can perform the computation on the bands themselves, which will apply the computation to each pixel-pairs.

In [ ]:
def calc_ndvi(nir, red):
    ndvi = (nir - red) / (nir + red)
    return ndvi

ndvi = calc_ndvi(nirf, redf)
plt.figure(figsize=[20, 20])
plt.imshow(ndvi, cmap='RdYlGn')
plt.colorbar()
plt.show()

The value range of an NDVI is -1 to 1. Negative values of NDVI (values approaching -1) correspond to water. Values close to zero (-0.1 to 0.1) generally correspond to barren areas of rock, sand, or snow. Low, positive values represent shrub and grassland (approximately 0.2 to 0.4), while high values indicate temperate and tropical rainforests (values approaching 1).

---

## Summary exercise on raster data management

The `LC08_L1TP_188027_20180501_20180516_01_T1.tif` file is a Landsat 8 satellite image from the same territory as the previous image, but acquired on 2018 May 1, so ca. 2 years earlier.  
Download: https://gis.inf.elte.hu/files/public/landsat-budapest-2018 (1.4 GB)

In [ ]:
budapest_2018 = rasterio.open('LC08_L1TP_188027_20180501_20180516_01_T1.tif')

**Task 1:** Calculate the NDVI for the 2018 Landsat satellite image.

**Task 2:** Compute the NDVI difference of the time interval and visaulize it.